# Accessibility of Ireland's mass vaccine centres

Motivation: there are 37 mass vaccination centres in Ireland. This algorithm aims to measure how accessible they are by public transport.

# Part 1: importing the data

The data for this analysis is stored in 51 different folders, each containing the GTFS files for a different operator. The coordinates for the vaccination centres are also stored in a separate file using a similar format to the GTFS files.

For the algorithm to work, the following GTFS files need to be imported as DataFrames, keeping the named columns:

1. stops.txt: stop_id, stop_name, stop_lat, stop_lon
2. Vaccine_Hubs.txt: acility, facility_lat, facility_lon
3. stop_times.txt:trip_id, arrival_time, departure_time, stop_id, stop_sequence, pickup_type, drop_off_type
4. trips.txt
5. calendar.txt

In [ ]:
%reset

In [ ]:
from os import walk
import pandas as pd
import numpy as np
from haversine import haversine_vector
import time

#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)


In [ ]:
gtfs_folder = 'GTFS Files'
operators = next(walk(gtfs_folder))[1]

MAX_TRANSFER_DIST = 2.0
MAX_WALK_DIST = 5.0
WALKING_SPEED = 5.0
WALK_EFFORT_FACTOR = 0.5

# Step 1a: import stop location data and identify transfer stops

In [90]:
def import_stops():
    stops = []

    for operator in operators:
        file = gtfs_folder + '\\' + operator + '\\' + 'stops.txt'
        df = pd.read_csv(file)[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
        stops.append(df)

    stops = pd.concat(stops).drop_duplicates('stop_id')  # some stops are used by multiple operators with the same ID, so will be duplicated
    stops['merge_key'] = 1
    print("Number of stops =", len(stops.index))

    transfer_stops = find_nearby_stops(stops)
    transfer_stops = transfer_stops[transfer_stops.transfer_distance < MAX_TRANSFER_DIST]
    transfer_stops['transfer_walk_time'] = transfer_stops['transfer_distance'] / WALKING_SPEED
    transfer_stops = transfer_stops[['stop_id_x', 'stop_id_y', 'transfer_walk_time']]
    return stops, transfer_stops

def find_nearby_stops(stops):
    t1 = time.time()
    ## Returns a dataframe containing all pairs of stops that are within walking distance of each other
    ## This function works by dividing the region into cells, each containing 0.1 degrees of longitude and latitude
    ## Stops are paired if they are in the same or neighbouring cell (including diagonal neighbours)    
    ## For Ireland, 0.1 degrees longitude is about 6.6 kms, while 0.1 degrees latitude is about 11.1 kms
    ## So this function will identify every pair of stops that are less than 6.6 kms apart
        
    decimals = 1
    round_precision = 0.1  # degrees longitude/latitude
    middle = round_precision / 2
    
    rounded_stops = stops.copy()
    
    ## lat_ceil, lat_floor, lon_ceil, lon_floor are the lat/lon values of the four edges of the cell
    rounded_stops['lat_ceil'] = rounded_stops['stop_lat'] + middle
    rounded_stops['lat_ceil'] = rounded_stops['lat_ceil'].round(decimals)

    rounded_stops['lon_ceil'] = rounded_stops['stop_lon'] + middle
    rounded_stops['lon_ceil'] = rounded_stops['lon_ceil'].round(decimals)

    rounded_stops['lat_floor'] = rounded_stops['lat_ceil'] - round_precision
    rounded_stops['lon_floor'] = rounded_stops['lon_ceil'] - round_precision
    
        
    cell_edges = []  
    ## This list will contain 4 DataFrames, one for each of the four corners of the cells
    ## These DataFrames will be concatenated into a single DataFrame
    ## Each stop in this DataFrame will have 4 rows, indicating the locations of the 4 corners of its cell
    ## By merging this DataFrame with itself, we will get every pair of stops that are in neighbouring cells    
    
    for lat_round in ['ceil', 'floor']:
        for lon_round in ['ceil', 'floor']:
            df = rounded_stops[['stop_id', 'lat_' + lat_round, 'lon_' + lon_round, 'stop_lat', 'stop_lon']]
            df = df.rename(columns={'lat_' + lat_round: 'rounded_lat',
                                    'lon_' + lon_round: 'rounded_lon'})
            
            cell_edges.append(df)    
    
    rounded_stops = pd.concat(cell_edges)    
    rounded_stops_copy = rounded_stops.copy()
    
    nearby_stops = pd.merge(rounded_stops, rounded_stops_copy, on=['rounded_lat', 'rounded_lon'])
    
    nearby_stops = nearby_stops.drop_duplicates(['stop_id_x', 'stop_id_y']) # Stop pairs in cells with multiple shared vertices will appear multiple times
    stop_x_coords = nearby_stops[['stop_lat_x', 'stop_lon_x']].values
    stop_y_coords = nearby_stops[['stop_lat_y', 'stop_lon_y']].values

    nearby_stops['transfer_distance'] = haversine_vector(stop_x_coords, stop_y_coords)

    del stop_x_coords, stop_y_coords

    nearby_stops = nearby_stops.sort_values('transfer_distance')
    
    return nearby_stops


def find_nearby_stops2(stops):
    t1 = time.time()
    ## Returns a dataframe containing all pairs of stops that are within walking distance of each other
    ## This function works by dividing the region into cells, each containing 0.1 degrees of longitude and latitude
    ## Stops are paired if they are in the same or neighbouring cell (including diagonal neighbours)
    ## For Ireland, 0.1 degrees longitude is about 6.6 kms, while 0.1 degrees latitude is about 11.1 kms
    ## So this function will identify every pair of stops that are less than 6.6 kms apart
        
    decimals = 1
    round_precision = 0.1  # degrees longitude/latitude
    middle = round_precision / 2

    rounded_stops = stops.copy()
    
    print(rounded_stops.columns)
    ## lat_ceil, lat_floor, lon_ceil, lon_floor are the lat/lon values of the four edges of the cell
    rounded_stops['lat_ceil'] = rounded_stops['stop_lat'] + middle
    rounded_stops['lat_ceil'] = rounded_stops['lat_ceil'].round(decimals)

    rounded_stops['lon_ceil'] = rounded_stops['stop_lon'] + middle
    rounded_stops['lon_ceil'] = rounded_stops['lon_ceil'].round(decimals)

    rounded_stops['lat_floor'] = rounded_stops['lat_ceil'] - round_precision
    rounded_stops['lon_floor'] = rounded_stops['lon_ceil'] - round_precision

    rounded_stops_copy = rounded_stops.copy()
    
    ## There are 9 cases to consider, corresponding to the 9 cells in a 3x3 grid where a nearby stop could be located
    ## df1: both stops in same cell => (lat_ceil, lon_ceil)_x = (lat_ceil, lon_ceil)_y
    ## df2a/b: cells are north/south neighbours => (lat_ceil, lon_ceil)_x = (lat_floor, lon_ceil)_y
    ## df3a/b: cells are east/west neighbours => (lat_ceil, lon_ceil)_x = (lat_ceil, lon_floor)_y
    ## df4a/b: cells are NE/SW diagonal neighbours => (lat_ceil, lon_ceil)_x = (lat_floor, lon_floor)_y
    ## df5a/b: cells are NW/SE diagonal neighbours => (lat_ceil, lon_floor)_x = (lat_floor, lon_ceil)_y

    
    df1 = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_ceil', 'lon_ceil'], right_on=['lat_ceil', 'lon_ceil'])
    
    df2a = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_ceil', 'lon_ceil'], right_on=['lat_floor', 'lon_ceil'])
    df2b = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_floor', 'lon_ceil'], right_on=['lat_ceil', 'lon_ceil'])

    df3a = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_ceil', 'lon_ceil'], right_on=['lat_ceil', 'lon_floor'])
    df3b = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_ceil', 'lon_floor'], right_on=['lat_ceil', 'lon_ceil'])

    df4a = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_ceil', 'lon_ceil'], right_on=['lat_floor', 'lon_floor'])
    df4b = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_floor', 'lon_floor'], right_on=['lat_ceil', 'lon_ceil'])

    df5a = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_ceil', 'lon_floor'], right_on=['lat_floor', 'lon_ceil'])
    df5b = pd.merge(rounded_stops, rounded_stops_copy, left_on=['lat_floor', 'lon_ceil'], right_on=['lat_ceil', 'lon_floor'])

    dfs = [df1, df2a, df2b, df3a, df3b, df4a, df4b, df5a, df5b]

    nearby_stops = pd.concat(dfs)

    nearby_stops = nearby_stops[['stop_id_x', 'stop_lat_x', 'stop_lon_x', 
                                 'stop_id_y', 'stop_lat_y', 'stop_lon_y']]

    del df1, df2a, df2b, df3a, df3b, df4a, df4b, df5a, df5b, dfs, rounded_stops, rounded_stops_copy
    
    stop_x_coords = nearby_stops[['stop_lat_x', 'stop_lon_x']].values
    stop_y_coords = nearby_stops[['stop_lat_y', 'stop_lon_y']].values

    nearby_stops['transfer_distance'] = haversine_vector(stop_x_coords, stop_y_coords)

    del stop_x_coords, stop_y_coords

    nearby_stops = nearby_stops.sort_values('transfer_distance')
    return nearby_stops

stops, transfer_stops = import_stops()

Number of stops = 12614
SSSSSSSSSSS


In [91]:
transfer_stops

,stop_id_x,stop_id_y,transfer_walk_time
0,700000015422,700000015422,0.000000
25098649,8450B3314701,8450B3314701,0.000000
751312,8220DB000522,8220DB000522,0.000000
25098602,8450B3314601,8450B3314601,0.000000
25098555,8450B3314501,8450B3314501,0.000000
...,...,...,...
10665097,8260B1038901,826000094,0.399998
25051754,8440B352541,844000056,0.399999
25078034,844000056,8440B352541,0.399999
175132,8220B123501,8220DB001515,0.400000


# Step 1b: import vaccine centre location data and identify nearby stops

In [92]:
vax_hubs = pd.read_csv('Vaccine_Hubs.txt')
vax_hubs['merge_key'] = 1

vax_hub_stops = pd.merge(stops, vax_hubs, on='merge_key')
stop_coords = vax_hub_stops[['stop_lat', 'stop_lon']].values
vax_hub_coords = vax_hub_stops[['Facility Latitude', 'Facility Longitude']].values

vax_hub_stops['stop_to_vax_hub_distance'] = haversine_vector(stop_coords, vax_hub_coords)
vax_hub_stops['stop_to_vax_hub_time'] = vax_hub_stops['stop_to_vax_hub_distance'] / WALKING_SPEED
vax_hub_stops = vax_hub_stops.sort_values('stop_to_vax_hub_distance')
vax_hub_stops = vax_hub_stops.drop_duplicates('stop_id')
vax_hub_stops

,stop_id,stop_name,stop_lat,stop_lon,merge_key,Facility Latitude,Facility Longitude,County,Facility,Address,stop_to_vax_hub_distance,stop_to_vax_hub_time
266744,8220DB007571,"DCU Helix, stop 7571",53.386776,-6.258725,1,53.386560,-6.259032,Dublin,Helix Theatre DCU,DCU Santry,0.031458,0.006292
79073,8370B2368501,"Eglinton Street, stop 236851",51.897310,-8.464863,1,51.897136,-8.465482,Cork,City Hall Cork,"City Hall, Anglesea Street, Cork city",0.046698,0.009340
74483,8360B603331,"West County Hotel, stop 603331",52.831752,-8.980457,1,52.831327,-8.980353,Clare,West County Hotel,"Limerick Road, Ennis",0.047821,0.009564
410200,828000017,Old Post Office Portlaoise,53.034486,-7.302756,1,53.035026,-7.302458,Laois,Midlands Park Hotel,"Jessop St., Portlaoise, Co Laois",0.063307,0.012661
410163,828000016,Old Post Office Portlaoise,53.034460,-7.302905,1,53.035026,-7.302458,Laois,Midlands Park Hotel,"Jessop St., Portlaoise, Co Laois",0.069738,0.013948
...,...,...,...,...,...,...,...,...,...,...,...,...
161935,8470B5304301,"Ballyconneely, stop 530431",53.431157,-10.075430,1,53.843820,-9.239317,Mayo,Breaffy House Resort,"Breaffy, Castlebar, Co Mayo",71.720747,14.344149
359921,700000014719,"Belfast City Centre, Glengall Street",54.595088,-5.937195,1,53.967790,-6.387424,Louth,Fairways Hotel,"Dublin Rd, Haggardstown, Dundalk, Co Louth",75.627856,15.125571
22,700000015422,"Belfast, Europa Bus Centre",54.595054,-5.936268,1,53.967790,-6.387424,Louth,Fairways Hotel,"Dublin Rd, Haggardstown, Dundalk, Co Louth",75.647670,15.129534
456750,gen:31400:8239:0:1,"Belfast City Centre, Jury's Inn",54.596260,-5.934783,1,53.967790,-6.387424,Louth,Fairways Hotel,"Dublin Rd, Haggardstown, Dundalk, Co Louth",75.808460,15.161692


# Step 1c: import data about trip times from the GTFS files

In [96]:
def import_stop_times():
    stop_times_dfs = []
    
    for operator in operators:
        stop_times = pd.read_csv(gtfs_folder + '\\' + operator + '\\' + 'stop_times.txt')
        stop_times['operator'] = operator
        stop_times_dfs.append(stop_times)
    
    stop_times = pd.concat(stop_times_dfs)
    stop_times['departure_time'] = pd.to_timedelta(stop_times['departure_time'])
    stop_times['arrival_time'] = pd.to_timedelta(stop_times['arrival_time'])
    
    stop_times['departure_time_hrs'] = stop_times['departure_time'].dt.total_seconds() / (60 * 60)
    stop_times['arrival_time_hrs'] = stop_times['arrival_time'].dt.total_seconds() / (60 * 60)

    stop_times = stop_times.drop_duplicates(['trip_id', 'stop_sequence'])
    print("Number of trip stops =", len(stop_times.index))
    return stop_times

def import_trip_data():
    trip_dfs = []
    todays_date = 20210225
    
    for operator in operators:
        calendar = pd.read_csv(gtfs_folder + '\\' + operator + '\\' + 'calendar.txt')
        calendar = calendar[(calendar['end_date'] > todays_date) & 
                            (calendar['start_date'] < todays_date)]
        
        trips = pd.read_csv(gtfs_folder + '\\' + operator + '\\' + 'trips.txt')
        trips = pd.merge(trips, calendar, on='service_id')
        trips['operator'] = operator
        trip_dfs.append(trips)
    
    trips = pd.concat(trip_dfs)

    del calendar, trip_dfs

    print("Number of trips =", len(trips.index))    
    return trips

stop_times = import_stop_times()
trip_data = import_trip_data()
stop_times = pd.merge(stop_times, trip_data, on='trip_id')
stop_times

Number of trip stops = 6142633
Number of trips = 128090


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,operator_x,...,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,operator_y
0,9.daily.1-700-y11-2.1.I,0 days 01:25:00,0 days 01:25:00,8240000548,1,NaN,0.0,1.0,0.00,google_transit_aircoach,...,1,1,1,1,1,1,1,20201221,20211221,google_transit_aircoach
1,9.daily.1-700-y11-2.1.I,0 days 01:45:00,0 days 01:45:00,8220DB000047,2,NaN,1.0,0.0,9651.30,google_transit_aircoach,...,1,1,1,1,1,1,1,20201221,20211221,google_transit_aircoach
2,9.daily.1-700-y11-2.1.I,0 days 01:55:00,0 days 01:55:00,8220DB000272,3,NaN,1.0,0.0,11379.45,google_transit_aircoach,...,1,1,1,1,1,1,1,20201221,20211221,google_transit_aircoach
3,9.daily.1-700-y11-2.1.I,0 days 01:58:00,0 days 01:58:00,8220DB000273,4,NaN,1.0,0.0,11867.14,google_transit_aircoach,...,1,1,1,1,1,1,1,20201221,20211221,google_transit_aircoach
4,9.daily.1-700-y11-2.1.I,0 days 02:00:00,0 days 02:00:00,8220DB004530,5,NaN,1.0,0.0,12957.82,google_transit_aircoach,...,1,1,1,1,1,1,1,20201221,20211221,google_transit_aircoach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3936421,13.MF-BH.14-WX2-y11-3.9.O,0 days 15:29:00,0 days 15:29:00,834000047,21,NaN,0.0,0.0,9012.02,google_transit_wexfordbus,...,1,1,1,1,1,0,0,20210104,20220104,google_transit_wexfordbus
3936422,13.MF-BH.14-WX2-y11-3.9.O,0 days 15:30:00,0 days 15:30:00,8340B3316201,22,NaN,0.0,0.0,9289.60,google_transit_wexfordbus,...,1,1,1,1,1,0,0,20210104,20220104,google_transit_wexfordbus
3936423,13.MF-BH.14-WX2-y11-3.9.O,0 days 15:31:00,0 days 15:31:00,834LL10364,23,NaN,0.0,0.0,9739.56,google_transit_wexfordbus,...,1,1,1,1,1,0,0,20210104,20220104,google_transit_wexfordbus
3936424,13.MF-BH.14-WX2-y11-3.9.O,0 days 15:32:00,0 days 15:32:00,834000055,24,NaN,0.0,0.0,9957.11,google_transit_wexfordbus,...,1,1,1,1,1,0,0,20210104,20220104,google_transit_wexfordbus


# Step 1d: Select a day of the week for analysis

In [97]:
earliest_departure = 13
latest_arrival = 23

days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
day_index = 5 # Saturday
day = days[day_index]
prev_day = days[(day_index-1) % 7]

daytime_stop_times = stop_times[(stop_times.departure_time_hrs > earliest_departure) &
                                (stop_times.arrival_time_hrs < latest_arrival)]
daytime_stop_times = daytime_stop_times[daytime_stop_times[day] == 1]

## We also need to account for trips where the dep/arr times are over 24 hours
prev_daytime_stop_times = stop_times[(stop_times.departure_time_hrs > 24 + earliest_departure) &
                                     (stop_times.arrival_time_hrs < 24 + latest_arrival)]
prev_daytime_stop_times = prev_daytime_stop_times[prev_daytime_stop_times[prev_day] == 1]
prev_daytime_stop_times['departure_time_hrs'] -= 24
prev_daytime_stop_times['arrival_time_hrs'] -= 24

daytime_stop_times = pd.concat([daytime_stop_times, prev_daytime_stop_times])
daytime_stop_times = daytime_stop_times[['trip_id', 'stop_id', 'stop_sequence', 'pickup_type',
                                         'drop_off_type', 'departure_time_hrs', 'arrival_time_hrs']]

arrivals = daytime_stop_times[daytime_stop_times['drop_off_type']==0][['trip_id', 'stop_id', 'stop_sequence', 'arrival_time_hrs']]
departures = daytime_stop_times[daytime_stop_times['pickup_type']==0][['trip_id', 'stop_id', 'stop_sequence', 'departure_time_hrs']]

daytime_stop_times

,trip_id,stop_id,stop_sequence,pickup_type,drop_off_type,departure_time_hrs,arrival_time_hrs
66,35.daily.1-700-y11-2.1.I,8240000548,1,0.0,1.0,19.416667,19.416667
67,35.daily.1-700-y11-2.1.I,8220DB000047,2,1.0,0.0,19.750000,19.750000
68,35.daily.1-700-y11-2.1.I,8220DB000272,3,1.0,0.0,19.916667,19.916667
69,35.daily.1-700-y11-2.1.I,8220DB000273,4,1.0,0.0,19.966667,19.966667
70,35.daily.1-700-y11-2.1.I,8220DB004530,5,1.0,0.0,20.000000,20.000000
...,...,...,...,...,...,...,...
3779185,1340.TA.99-13-r11-1.11.O,822GIR0141,23,0.0,0.0,19.966667,19.950000
3779186,1340.TA.99-13-r11-1.11.O,824G003874,24,0.0,0.0,19.983333,19.983333
3779187,1340.TA.99-13-r11-1.11.O,824GIR0140,25,0.0,0.0,20.033333,20.016667
3779188,1340.TA.99-13-r11-1.11.O,824GIR0024,26,0.0,0.0,20.066667,20.050000


# Part 2: the routing algorithm

Next we need to find the journey time to every transit stop from a vaccine centre.

The algorithm used here is a breadth first search, where all possible trips are saved unless they are proven to be suboptimal.

### What is a sub-optimal trip?

In this case, a trip is sub-optimal if there is another trip which leaves a vaccine centre later and arrives at the same stop earlier.

There are two important caveats to note:

Firstly, the departure time used is adjusted to account for aversion to large amounts of walking. The formula is:

    adj_journey_dep_time = journey_dep_time - WALK_EFFORT_FACTOR * total_walk_time

where WALK_EFFORT_FACTOR is a number which increases with aversion to walking. Effectively, this gives a slight preference to longer journeys with less walking.

A second adjustment is also made to trips that are exclusively on foot. This adjustment reflects the fact that people are unlikely to take a public transport journey for a trip that can be walked in a comparable amount of time, even if the public transport option is a little faster. For this algorithm, a public transport option would only be chosen over walking if it is at least 5 minutes faster end to end. This eliminates hyperlocal transit journeys.

### How are the trips stored?

The trips are saved in a DataFrame, with each row representing a single trip. The columns of the DataFrame tell us the important information about the trip such as the departure and arrival times, the vaccination centre used, the amount of walking, and the trip_ids and stop_ids along the way.

The columns in the DataFrame are as follows:

1. departure_facility
2. total_walk_time: includes all time spent walking at the start and at transfers, but not waiting time at transfers
3. journey_dep_time: This is the time of departure from the vaccination centre
4. journey_arrival_time: This is the time of arrival in the final stop
5. adj_journey_dep_time: This is the adjusted departure time, accounting for aversion to walking, and used to identify sub-optimal trips
6. final_stop_id
7. walk_1_time: Time to walk from the vaccination centre to the first stop
8. stop_1a_id: First departure stop
9. trip_1_id
10. trip_1_departure_time
11. trip_1_arrival_time
12. stop_1b_id: First arrival stop

Columns 1-6 relate to the overall journey

Columns 7-12 give the details of the first leg

Each subsequent leg of the journey will use the same format as columns 7-12, but with a different number.

Note that walk_2_time, walk_3_time, etc only measure the actual walk time, not the total time from arriving in one stop to departing from the next.

### How are sub-optimal trips identified?

Given a DataFrame as described above, how do we identify and remove the rows coresponding to sub-optimal trips?

Walkable trips can be identified by merging the DataFrame with vax_hub_stops on final_stop_id. From here, we can easily compare the trip time with the journey time on foot for each row, keeping only rows where the transit journey time is at least 5 minutes faster.

For late depart/early arrive, the algorithm is a little more involved. The solution used here starts by sorting the DataFrame by final_stop_id and journey_arrival_time. This ensures that all rows with the same final_stop_id are clustered together, with arrival times increasing. The first occurence of every final_stop_id is retained, since this corresponds to the earliest known arrival. Other rows are retained only if their adjusted departure time is later than the adjusted departure time for the previous row.

##### Sub-optimal departures
Throughout this algorithm, there will be several times when the trips are only partially computed. One case is when the possible departures have been identified, but not the corresponding arrivals. It is very important to note that the late depart/early arrive process cannot be used in this situation, since it is not sub-optimal to skip a departure to catch a bus to a different place. However, there is a variant of this algorithm which does work in this situation:

Under this process, a trip can be identified as sub-optimal if there is another trip which departs the vaccination centre later and joins the same bus/train journey earlier in the vehicle's journey. This process is extremely useful for managing transit routes which run alongside one another. The problem here is that without this check, a tranfer between parallel bus routes will be replicated at every stop along the parallel section. In Dublin, this can easily be dozens of stops with an effectively identical transfer. This process reduces all of those down to one.

### Bringing it all together

The process for the full algorithm is as follows:
#### 1. Find stops within walking distance of a given vaccination centre
    This information can be pulled from vax_hub_stops, by filtering stop_to_vax_hub_distance
    For this analysis, the maximum walking distance is set to 5 kms
    There are three columns we need to retain: departure_facility, stop_id and stop_to_vax_hub_time
#### 2. Find all departures from those stops
    We do this by merging the DataFrame from the previous step with the departures DataFrame on the stop_id column
    This will give us a DataFrame containing the first 10 columns from above
#### 3. Find arrivals corresponding to those departures
    The next step is to merge the DataFrame with the arrivals DataFrame, using the trip_id as the merge key
    We need to do a further check to ensure the arrival stop occurs after the departure stop. Conveniently, the stops are
    numbered for each trip, so we simply need to keep the rows where the arrival stop number is bigger than the departure 
    stop number.
    We now have all 12 columns corresponding to a single trip journey
#### 4. Remove sub-optimal trips
    We use the processes described above to remove sub-optimal rows
#### 5. Find transfer stops and remove sub-optimal trips
    This involves merging the DataFrame with transfer_stops on final_stop_id.
    Some trips will become sub-optimal due to the presence of a superior trip at a nearby stop.
#### 6. Identify departures, as in step 2 and remove sub-optimal departures
    Note that this merge gets very conputationally intenstive, and is immediately followed by a significant optimisation, as discussed above.
    Therefore, it is generally faster to perform the merge and optimisation in chunks.
#### 7. Identify arrivals as in step 3 and remove sub-optimal trips 
#### 8. Repeat steps 5-7 for each subsequent leg   

In [98]:
vc = transfer_stops['stop_id_x'].value_counts()
rural_stops = vc[vc < 5].index
print("Number of rural stops: ", len(rural_stops))

limited_transfers = transfer_stops[(transfer_stops.transfer_walk_time < 0.2) | 
                                   (transfer_stops.stop_id_x.isin(rural_stops)) | 
                                   (transfer_stops.stop_id_y.isin(rural_stops))]
limited_transfers = limited_transfers[['stop_id_x', 'stop_id_y', 'transfer_walk_time']]
limited_transfers

Number of rural stops:  2950


,stop_id_x,stop_id_y,transfer_walk_time
0,700000015422,700000015422,0.000000
25098649,8450B3314701,8450B3314701,0.000000
751312,8220DB000522,8220DB000522,0.000000
25098602,8450B3314601,8450B3314601,0.000000
25098555,8450B3314501,8450B3314501,0.000000
...,...,...,...
10851212,8470PB000860,8470PB000857,0.398976
23059782,8360B336431,836GIR0003,0.399008
23062190,836GIR0003,8360B336431,0.399008
25951363,853000334,853000120,0.399749


In [104]:
def eliminate_early_depart_late_arrive(trips, stop_id_col_name='final_stop_id',
                                       dep_time_col_name='adj_journey_dep_time', 
                                       arr_time_col_name='journey_arrival_time'):
    ## This function eliminates trips where there is another trip which leaves later and arrives at the same place earlier
    ## Start by sorting the df of trips by stop_id, arrival_time, and adj_dep_time
    
    trips = trips.sort_values([stop_id_col_name, arr_time_col_name, dep_time_col_name],
                               ascending=[True, True, False])

    ## The following loop retains the first occurence of every stop_id (i.e. the one with the latest departure time)
    ## Subsequent occurences are ordered in decreasing departure time, 
    ## and are retained only of their arrival time is earlier than the one above
    
    ## This needs to be repeated until all early depart/late arrive trips have been removed
    
    while True:
        initial_len = len(trips.index)
        trips = trips[(~trips[stop_id_col_name].duplicated()) | 
                      (trips[dep_time_col_name].diff() > 0)]
        new_len = len(trips.index)

        if initial_len == new_len:
            return trips

def remove_walkable_trips(trips, threshold=0.1):
    ## Removes any trips that can be completed faster on foot
    ## Also removes trips that take longer on foot up to a given threshold (in hours)
    
    trips = pd.merge(trips, vax_hub_stops, left_on='final_stop_id', right_on='stop_id')
    direct_to_stop_time = (1 + WALK_EFFORT_FACTOR) * trips['stop_to_vax_hub_time']
    transit_journey_time = trips['journey_arrival_time'] - trips['adj_journey_dep_time']
    trips = trips[direct_to_stop_time > transit_journey_time + threshold]
    trips = trips.drop(columns=vax_hub_stops.columns)
    return trips

def eliminate_back_track(current_trips, extended_terminated_trips):
    current_trips['terminated'] = False
    for df in extended_terminated_trips:
        df['terminated'] = True
        current_trips = pd.concat([current_trips, df])
    
    current_trips = eliminate_early_depart_late_arrive(current_trips)
    current_trips = current_trips[~current_trips.terminated]
    
    return current_trips

In [100]:
def compute_first_legs(): 
    local_stops = vax_hub_stops[vax_hub_stops.stop_to_vax_hub_distance < MAX_WALK_DIST]
    local_stops = local_stops[['stop_id', 'stop_to_vax_hub_time', 'Facility']]    
        
    first_legs = pd.merge(local_stops, departures, on='stop_id')
    first_legs = first_legs.rename(columns={'departure_time_hrs': 'leg_1_departure_time', 
                                            'stop_id': 'stop_1a_id', 
                                            'stop_to_vax_hub_time': 'walk_1_time',
                                            'Facility': 'departure_facility'})
    
    first_legs['journey_dep_time'] = first_legs['leg_1_departure_time'] - first_legs['walk_1_time']
    first_legs['total_walk_time'] = first_legs['walk_1_time']
    first_legs['adj_journey_dep_time'] = first_legs['journey_dep_time'] - WALK_EFFORT_FACTOR * first_legs['total_walk_time']
    
    first_legs = pd.merge(first_legs, arrivals, on='trip_id')

    first_legs = first_legs[first_legs.stop_sequence_y > first_legs.stop_sequence_x]
    
    first_legs = first_legs.rename(columns={'trip_id': 'leg_1_trip_id',
                                            'arrival_time_hrs': 'leg_1_arrival_time',
                                            'stop_id': 'stop_1b_id'})

    first_legs['journey_arrival_time'] = first_legs['leg_1_arrival_time']
    first_legs['final_stop_id'] = first_legs['stop_1b_id']
    
    first_legs = first_legs[['departure_facility', 'total_walk_time', 'journey_dep_time',
                             'journey_arrival_time', 'adj_journey_dep_time', 'final_stop_id', 
                             'walk_1_time', 'stop_1a_id', 'leg_1_trip_id', 
                             'leg_1_departure_time', 'leg_1_arrival_time', 'stop_1b_id']]
    
    first_legs = eliminate_early_depart_late_arrive(first_legs)
    first_legs = remove_walkable_trips(first_legs)
    
    return first_legs

first_legs = compute_first_legs()

TTT:  0.6044955253601074 3.7562146186828613
4.478371858596802 0.3169722557067871


In [105]:
def identify_transfer_stops(trips, connected_stops, leg_number):
    ## Extends the trips DataFrame by finding all stops within walking distance of the current stops
    
    ## connected_stops.columns = ['stop_id_x', 'stop_id_y', 'transfer_walk_time']
    
    trips = pd.merge(trips, connected_stops, left_on='final_stop_id', right_on='stop_id_x').drop(columns='stop_id_x')
    
    trips['final_stop_id'] = trips['stop_id_y']
    
    prev_arrival_time = 'leg_' + str(leg_number-1) + '_arrival_time'
    trips['journey_arrival_time'] = trips[prev_arrival_time] + trips['transfer_walk_time']
    
    trips['total_walk_time'] += trips['transfer_walk_time']
    trips['adj_journey_dep_time'] = trips['journey_dep_time'] - WALK_EFFORT_FACTOR * trips['total_walk_time']
    
    col_names = {'stop_id_y': 'stop_' + str(leg_number) + 'a_id',
                 'transfer_walk_time': 'walk_' + str(leg_number) + '_time'}
    trips = trips.rename(columns=col_names)

    trips = eliminate_early_depart_late_arrive(trips)
    
    trips = remove_walkable_trips(trips)
    
    return trips

def get_departures(trips, leg_number, min_transfer_time=0.1, chunk_size=None):
    if chunk_size == None:
        chunk_size = len(trips.index)
        
    lower_bound = 0
    dfs = []
    
    while lower_bound < len(trips.index):
        upper_bound = min(lower_bound + chunk_size, len(trips.index))
        
        df = trips[lower_bound:upper_bound]    
        df = pd.merge(df, departures, left_on='final_stop_id', right_on='stop_id').drop(columns='stop_id')
        df = df[df.departure_time_hrs > df.journey_arrival_time]
        df = df.sort_values(['trip_id', 'adj_journey_dep_time', 'stop_sequence'],
                            ascending=[False, False, True])

        duplicate_vals = True
        while duplicate_vals:
            initial_len = len(df.index)
            df = df[(~df['trip_id'].duplicated()) | 
                  (df['stop_sequence'].diff() < -1)]
            new_len = len(df.index)

            if initial_len == new_len:
                duplicate_vals = False
        
        dfs.append(df)
        lower_bound += chunk_size
    
    df = pd.concat(dfs)
    
    duplicate_vals = True
    while duplicate_vals:
        initial_len = len(df.index)
        df = df[(~df['trip_id'].duplicated()) | 
              (df['stop_sequence'].diff() < -1)]
        new_len = len(df.index)

        if initial_len == new_len:
            duplicate_vals = False
    
    df = df.rename(columns={'trip_id': 'leg_' + str(leg_number) + '_trip_id',
                            'departure_time_hrs': 'leg_' + str(leg_number) + '_departure_time'})
    
    return df

In [102]:
def get_arrivals(trips, leg_number, chunk_size=None):
    trip_col = 'leg_' + str(leg_number) + '_trip_id'
    
    arrival_time_col = 'leg_' + str(leg_number) + '_arrival_time'
    arrival_stop_col = 'stop_' + str(leg_number) + 'b_id'
    
    df = pd.merge(trips, arrivals, left_on='leg_' + str(leg_number) + '_trip_id', right_on='trip_id').drop(columns='trip_id')
    df = df[df.stop_sequence_y > df.stop_sequence_x]
    df = df.drop(columns=['stop_sequence_x', 'stop_sequence_y'])
    df = df.rename(columns={'arrival_time_hrs': arrival_time_col,
                            'stop_id': arrival_stop_col})
    df['journey_arrival_time'] = df[arrival_time_col]
    df['final_stop_id'] = df[arrival_stop_col]

    df = eliminate_early_depart_late_arrive(df)
    df = remove_walkable_trips(df)

    return df

In [110]:
legs = [first_legs]
terminated_trips = []


for leg_number in range(2, 6):
    leg_dep_stops = identify_transfer_stops(legs[-1], limited_transfers, leg_number)
    legs.append(leg_dep_stops)
    terminated_trips.append(leg_dep_stops)
    print(leg_number, len(leg_dep_stops.index))

    leg_departures = get_departures(legs[-1], leg_number, chunk_size=20000)
    legs.append(leg_departures)
    print(leg_number, len(legs[-1].index))

    leg_arrivals = get_arrivals(legs[-1], leg_number)
    print(leg_number, len(leg_arrivals.index))
    leg_arrivals = eliminate_back_track(leg_arrivals, terminated_trips)
    legs.append(leg_arrivals)
    print(leg_number, len(legs[-1].index))



TTT:  3.121650457382202 10.718326330184937
2 509480
2 31550
TTT:  0.3101694583892822 0.31914424896240234
2 126632
TTT:  1.3912758827209473 0.7988648414611816
2 60746
TTT:  2.2051544189453125 4.677961826324463
3 334466
3 31053
TTT:  0.29720354080200195 0.35505032539367676
3 113418
TTT:  2.075670003890991 1.092104434967041
3 34399
TTT:  1.1087119579315186 2.0592610836029053
4 206200
4 25703
TTT:  0.31054186820983887 0.3370983600616455
4 92678
TTT:  2.4635813236236572 1.5827605724334717
4 22621
TTT:  0.837775468826294 2.1829609870910645
5 133084
5 21912
TTT:  0.31615495681762695 0.4278550148010254
5 79329
TTT:  2.4104464054107666 1.6642510890960693
5 11744


In [113]:
legs[6]

,departure_facility,total_walk_time,journey_dep_time,journey_arrival_time,adj_journey_dep_time,final_stop_id,walk_1_time,stop_1a_id,leg_1_trip_id,leg_1_departure_time,...,leg_2_departure_time,stop_2b_id,leg_2_arrival_time,terminated,stop_3a_id,walk_3_time,leg_3_trip_id,leg_3_departure_time,stop_3b_id,leg_3_arrival_time
0,Fairways Hotel,0.106272,13.852061,15.383333,13.798925,700000000229,0.106272,8300B133581,1470845.14.10-168-e19-1.489.I,13.958333,...,14.500000,7000B156051,15.083333,False,7000B156051,0.000000,1470785.14.10-160-e19-1.440.I,15.250000,700000000229,15.383333
10,Letterkenny Institute of Technology,0.044849,15.331168,16.750000,15.308744,700000013460,0.035498,8530B1559601,1414937.7.10-32-e19-1.91.I,15.366667,...,16.416667,8530B1522701,16.500000,False,8530B152491,0.009351,1475896.14.10-494-e19-1.2049.I,16.683333,700000013460,16.750000
13,Aviva Stadium,0.055799,13.200615,15.333333,13.172716,700000014719,0.032718,822GIR0133,214.TA.99-13-r11-1.3.O,13.233333,...,13.266667,822GIR0025,13.333333,False,8220DB000407,0.023080,9.daily.8-400-y11-1.1.O,13.500000,700000014719,15.333333
14,Aviva Stadium,0.055799,14.200615,16.333333,14.172716,700000014719,0.032718,822GIR0133,217.TA.99-13-r11-1.3.O,14.233333,...,14.266667,822GIR0025,14.333333,False,8220DB000407,0.023080,10.daily.8-400-y11-1.1.O,14.500000,700000014719,16.333333
15,Aviva Stadium,0.055799,15.283948,17.333333,15.256049,700000014719,0.032718,822GIR0133,1005.TA.99-13-r11-1.10.O,15.316667,...,15.350000,822GIR0025,15.416667,False,8220DB000407,0.023080,11.daily.8-400-y11-1.1.O,15.500000,700000014719,17.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157071,Letterkenny Institute of Technology,0.104140,14.049661,17.583333,13.997592,gen:57402:7996:0:1,0.033672,8530B158221,2.Sat.3-931-y11-2.7.I,14.083333,...,15.166667,853000122,15.916667,False,853000121,0.000000,4.daily.49-954-y11-1.1.O,17.000000,gen:57402:7996:0:1,17.583333
157074,Letterkenny Institute of Technology,0.105214,14.049661,18.250000,13.997055,gen:57402:8178:0:1,0.033672,8530B158221,2.Sat.3-931-y11-2.7.I,14.083333,...,15.166667,853000122,15.916667,False,853000124,0.001074,6.Mo-Sa.49-955-y11-2.5.I,17.500000,gen:57402:8178:0:1,18.250000
157075,Letterkenny Institute of Technology,0.105214,14.049661,18.050000,13.997055,gen:57402:8179:0:1,0.033672,8530B158221,2.Sat.3-931-y11-2.7.I,14.083333,...,15.166667,853000122,15.916667,False,853000124,0.001074,6.Mo-Sa.49-955-y11-2.5.I,17.500000,gen:57402:8179:0:1,18.050000
157076,Letterkenny Institute of Technology,0.131984,15.414502,18.166667,15.348510,gen:57402:81:0:1,0.035498,8530B1559601,1414841.7.10-64-e19-1.174.I,15.450000,...,17.166667,853000360,17.833333,False,853000360,0.000000,9.Mo-Sa.3-957-y11-1.6.H,17.916667,gen:57402:81:0:1,18.166667


# Deprecated stuff

In [ ]:
def compute_first_legs_from_facility_dep(facility):
    ## This finds all trips that can be make with a single leg from the stops closest to a given vaccine centre
    
    local_stops = vax_hub_stops[vax_hub_stops.Facility == facility] # All stops that are closer to this facility than any other
    local_stops = local_stops[local_stops.stop_to_vax_hub_distance < MAX_WALK_DIST]
    local_stops = local_stops[['stop_id', 'stop_to_vax_hub_time']]
    
    
    first_legs_from_facility = pd.merge(local_stops, departures, on='stop_id')
    first_legs_from_facility = first_legs_from_facility.rename(columns={'departure_time_hrs': 'leg_1_departure_time', 
                                                                        'stop_id': 'stop_1a_id', 
                                                                        'stop_to_vax_hub_time': 'walk_1_time'})
    
    first_legs_from_facility = first_legs_from_facility.sort_values('walk_1_time')
    first_legs_from_facility = first_legs_from_facility.drop_duplicates('trip_id')  # some trips will appear multiple times, each starting from a different stop
    
    first_legs_from_facility['journey_dep_time'] = first_legs_from_facility['leg_1_departure_time'] - first_legs_from_facility['walk_1_time']
    first_legs_from_facility['total_walk_time'] = first_legs_from_facility['walk_1_time']
    first_legs_from_facility['adj_journey_dep_time'] = first_legs_from_facility['journey_dep_time'] - WALK_EFFORT_FACTOR * first_legs_from_facility['total_walk_time']
    
    t3 = time.time()
    first_legs_from_facility = pd.merge(first_legs_from_facility, arrivals, on='trip_id')
    first_legs_from_facility['departure_facility'] = facility
    '''
    first_legs_from_facility = first_legs_from_facility[first_legs_from_facility.stop_sequence_y > first_legs_from_facility.stop_sequence_x]
    print(str(len(first_legs_from_facility.index)) + ' +')

    first_legs_from_facility = first_legs_from_facility.rename(columns={'trip_id': 'leg_1_trip_id', 
                                                                        'arrival_time_hrs': 'leg_1_arrival_time', 
                                                                        'stop_id': 'stop_1b_id'})

    first_legs_from_facility['journey_arrival_time'] = first_legs_from_facility['leg_1_arrival_time']
    first_legs_from_facility['final_stop_id'] = first_legs_from_facility['stop_1b_id']
    

    first_legs_from_facility = first_legs_from_facility[['departure_facility', 'total_walk_time', 'journey_dep_time', 
                                                         'journey_arrival_time', 'adj_journey_dep_time', 'final_stop_id', 
                                                         'walk_1_time', 'stop_1a_id', 'leg_1_trip_id', 
                                                         'leg_1_departure_time', 'leg_1_arrival_time', 'stop_1b_id']]
    
    #first_legs_from_facility = eliminate_early_depart_late_arrive(first_legs_from_facility)
    #first_legs_from_facility = remove_walkable_trips(first_legs_from_facility)
    '''
    return first_legs_from_facility

def compute_first_legs_dep():
    t1 = time.time()
    first_legs = []

    facilities = vax_hub_stops['Facility'].unique()
    for facility in facilities:
        first_legs_from_facility = compute_first_legs_from_facility(facility)
        first_legs.append(first_legs_from_facility) 
    all_first_legs = pd.concat(first_legs)
    #all_first_legs = eliminate_early_depart_late_arrive(all_first_legs)
    #all_first_legs = remove_walkable_trips(all_first_legs)
    return all_first_legs


#first_legs_old = compute_first_legs()
#first_legs_old

In [ ]:
def find_nearby_stops2(stops):
    t1 = time.time()
    ## Returns a dataframe containing all pairs of stops that are within walking distance of each other
    ## This function works by dividing the region into cells, each containing 0.1 degrees of longitude and latitude
    ## Stops are paired if they are in the same or neighbouring cell (including diagonal neighbours)
    ## For Ireland, 0.1 degrees longitude is about 6.6 kms, while 0.1 degrees latitude is about 11.1 kms
    ## So this function will identify every pair of stops that are less than 6.6 kms apart
        
    decimals = 1
    round_precision = 0.1  # degrees longitude/latitude
    middle = round_precision / 2
    
    rounded_stops = stops.copy()
    
    ## lat_ceil, lat_floor, lon_ceil, lon_floor are the lat/lon values of the four edges of the cell
    rounded_stops['lat_ceil'] = rounded_stops['stop_lat'] + middle
    rounded_stops['lat_ceil'] = rounded_stops['lat_ceil'].round(decimals)

    rounded_stops['lon_ceil'] = rounded_stops['stop_lon'] + middle
    rounded_stops['lon_ceil'] = rounded_stops['lon_ceil'].round(decimals)

    rounded_stops['lat_floor'] = rounded_stops['lat_ceil'] - round_precision
    rounded_stops['lon_floor'] = rounded_stops['lon_ceil'] - round_precision
        
    cell_edges = []
    
    for lat_round in ['ceil', 'floor']:
        for lon_round in ['ceil', 'floor']:
            df = rounded_stops[['stop_id', 'lat_' + lat_round, 'lon_' + lon_round, 'stop_lat', 'stop_lon']]
            df = df.rename(columns={'lat_' + lat_round: 'rounded_lat',
                                    'lon_' + lon_round: 'rounded_lon'})
            
            cell_edges.append(df)
    
    
    print("NEXT")
    
    rounded_stops = pd.concat(cell_edges)
    print("CONCAT")
    
    rounded_stops_copy = rounded_stops.copy()
    
    print("MERGE START")
    nearby_stops = pd.merge(rounded_stops, rounded_stops_copy, on=['rounded_lat', 'rounded_lon'])
    print("END")
    
    nearby_stops = nearby_stops.drop_duplicates(['stop_id_x', 'stop_id_y'])
    t2 = time.time()
    print(t2-t1, len(nearby_stops.index))
    return nearby_stops
